In [1]:
import numpy as np
import pandas as pd
import networkx as nx

import os
import re

from scipy.sparse import csr_matrix

In [4]:
cora_citations = pd.read_csv("../data/cora/cites.tsv", delimiter="\t", header=None)

In [6]:
cora_citations.shape

(5429, 2)

In [12]:
?cora_citations.drop_duplicates()

In [13]:
G = nx.read_edgelist("../data/cora/cites.tsv", delimiter="\t", )

In [14]:
len(G)

2708

In [15]:
len(G.edges)

5278

In [16]:
nx.number_connected_components(G)

78

In [4]:
cora_label_df = pd.read_csv("../data/cora/paper.tsv", delimiter="\t", index_col=0)

In [7]:
cora_label_df

,class_label
paper_id,
35,Genetic_Algorithms
40,Genetic_Algorithms
114,Reinforcement_Learning
117,Reinforcement_Learning
128,Reinforcement_Learning
130,Reinforcement_Learning
164,Theory
288,Reinforcement_Learning
424,Rule_Learning


In [11]:
cora_label_df["class_label"].unique()

array(['Genetic_Algorithms', 'Reinforcement_Learning', 'Theory',
       'Rule_Learning', 'Case_Based', 'Probabilistic_Methods',
       'Neural_Networks'], dtype=object)

In [15]:
y = np.array([1,1,3,2,8])

In [31]:
from keras import backend as K
from keras import layers

In [32]:
?layers.BatchNormalization

In [30]:
?K.normalize_batch_in_training

In [27]:
K

NameError: name 'K' is not defined

In [16]:
y

array([1, 1, 3, 2, 8])

In [19]:
np.unique(y)

array([1, 2, 3, 8])

In [22]:
A = np.zeros((10, 5))

In [24]:
A[np.array([1,4,6]), :2]

array([[ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.]])

In [25]:
a = np.array([1, -1, -1, -1, 2, -1, -1, -1, 3, -1, -1, -1])

In [26]:
n = 3
step_size = len(a) / n
a[::step_size]

array([1, 2, 3])

In [9]:
map_=  {i : cora_label_df.loc[:,"class_label"].unique()[i] for i in range(7)}

In [10]:
map_

{0: 'Genetic_Algorithms',
 1: 'Reinforcement_Learning',
 2: 'Theory',
 3: 'Rule_Learning',
 4: 'Case_Based',
 5: 'Probabilistic_Methods',
 6: 'Neural_Networks'}

G = nx.read_gml("../data/facebook_graph.gml")

In [55]:
len(G)

4039

In [80]:
feature_files = sorted([f for f in os.listdir('../data/facebook/') if re.match("[0-9]*.feat$", f)])
feature_name_files = sorted([f for f in os.listdir('../data/facebook/') if re.match("[0-9]*.featnames$", f)])


In [214]:
nodes_to_remove

[u'3437', u'686', u'1912', u'3980']

In [93]:
feature_files, feature_name_files

(['0.feat',
  '107.feat',
  '1684.feat',
  '1912.feat',
  '3437.feat',
  '348.feat',
  '3980.feat',
  '414.feat',
  '686.feat',
  '698.feat'],
 ['0.featnames',
  '107.featnames',
  '1684.featnames',
  '1912.featnames',
  '3437.featnames',
  '348.featnames',
  '3980.featnames',
  '414.featnames',
  '686.featnames',
  '698.featnames'])

In [185]:
feature_dfs = [pd.read_csv("../data/facebook/{}".format(f), header=None, index_col=0, sep=" ") for f in feature_files]

In [186]:
feature_name_dfs = [pd.read_csv("../data/facebook/{}".format(f), header=None, index_col=0, sep=" ") for f in feature_name_files]

In [187]:
feature_name_maps = [feature_name_df.loc[:,3].to_dict() for feature_name_df in feature_name_dfs]

In [188]:
for feature_name_map, feature_df in zip(feature_name_maps, feature_dfs):
    feature_df.columns = [feature_name_map[i-1] for i in feature_df.columns]
#     print feature_df.duplicated()
#     feature_df.drop_duplicates()

In [235]:
X = pd.concat(feature_dfs, join="outer", axis=1, ).fillna(0)
X.index = [str(i) for i in X.index]

In [237]:
nodes_to_keep = [n for n in list(G) if n in X.index]
nodes_to_remove = [n for n in list(G) if n not in X.index]

In [238]:
nodes_to_remove

[u'3437', u'686', u'1912', u'3980']

In [239]:
for node in nodes_to_remove:
    X.loc[node, :] = 0

In [241]:
X = X.reindex(list(G.nodes))

In [242]:
X.shape

(4039, 2280)

In [243]:
X = csr_matrix(X.values, dtype=np.float32)

In [244]:
X

<4039x2280 sparse matrix of type '<type 'numpy.float32'>'
	with 39367 stored elements in Compressed Sparse Row format>

In [248]:
sp.sparse.save_npz("../data/facebook/features.npz", X)

In [123]:
circle_files = sorted([f for f in os.listdir('../data/facebook/') if re.match("[0-9]*.circles$", f)])

In [135]:
circle_files

['0.circles',
 '107.circles',
 '1684.circles',
 '1912.circles',
 '3437.circles',
 '348.circles',
 '3980.circles',
 '414.circles',
 '686.circles',
 '698.circles']

In [148]:
node_assignments = {n : [] for n in G.nodes()}
circle = 0
for circle_file in circle_files:
    with open("../data/facebook/{}".format(circle_file), "r") as f:
#         cf = circle_file.split(".")[0]
        for line in f:
            line_split = line.rstrip().split("\t")
            for node in line_split[1:]:
                node_assignments[node].append(circle)
            circle += 1

In [160]:
len(G)

4039

In [154]:
node_assignments

{u'2031': [61, 91],
 u'4026': [150],
 u'3724': [],
 u'4024': [145],
 u'4025': [152],
 u'4022': [],
 u'4023': [148],
 u'4020': [148],
 u'4021': [150],
 u'643': [163],
 u'4028': [148],
 u'4029': [153],
 u'344': [4],
 u'345': [15],
 u'346': [],
 u'347': [15],
 u'340': [15],
 u'341': [15],
 u'342': [15],
 u'343': [6, 19],
 u'3997': [148],
 u'3996': [148],
 u'3995': [158],
 u'3994': [148],
 u'348': [27, 159, 160, 161],
 u'349': [129, 131],
 u'3991': [148],
 u'3990': [152],
 u'1343': [],
 u'2919': [47],
 u'1296': [],
 u'585': [],
 u'2318': [61, 87, 91],
 u'2319': [88, 92],
 u'2316': [65, 91],
 u'2317': [91],
 u'2314': [91],
 u'2315': [88],
 u'2312': [86, 92],
 u'2313': [91],
 u'2310': [61, 91],
 u'2311': [70],
 u'2184': [70],
 u'298': [0, 11],
 u'299': [16],
 u'296': [],
 u'297': [15],
 u'294': [15, 17],
 u'295': [4],
 u'292': [9],
 u'293': [20],
 u'290': [15],
 u'291': [15],
 u'3773': [],
 u'3772': [127],
 u'3771': [98, 110],
 u'3770': [110],
 u'3777': [110],
 u'3776': [],
 u'3775': [103, 1

In [157]:
Y = np.zeros((len(node_assignments), circle))

In [173]:
for i in range(len(G)):
    assignments = node_assignments[list(G)[i]]
    for assignment in assignments:
        Y[i, assignment] = 1

In [176]:
Y = csr_matrix(Y, dtype=np.float32)

In [177]:
Y

<4039x193 sparse matrix of type '<type 'numpy.float32'>'
	with 4233 stored elements in Compressed Sparse Row format>

In [246]:
import scipy as sp

In [247]:
sp.sparse.save_npz("../data/facebook/circle_labels.npz", Y)

In [2]:
hippie_df = pd.read_csv("../data/hippie/hippie_current.txt", sep="\t", header=None)

In [15]:
hippie_df

,0,1,2,3,4,5
0,AL1A1_HUMAN,216,AL1A1_HUMAN,216,0.76,"experiments:in vivo,Two-hybrid;pmids:12081471,..."
1,ITA7_HUMAN,3679,ACHA_HUMAN,1134,0.73,"experiments:in vivo,Affinity Capture-Western,a..."
2,NEB1_HUMAN,55607,ACTG_HUMAN,71,0.65,"experiments:in vitro,in vivo;pmids:9362513,120..."
3,SRGN_HUMAN,5552,CD44_HUMAN,960,0.63,"experiments:in vivo;pmids:9334256,16189514,167..."
4,GRB7_HUMAN,2886,ERBB2_HUMAN,2064,0.90,"experiments:in vitro,in vivo,Reconstituted Com..."
5,PAK1_HUMAN,5058,ERBB2_HUMAN,2064,0.73,"experiments:in vivo,Affinity Capture-Western,a..."
6,DLG4_HUMAN,1742,ERBB2_HUMAN,2064,0.87,"experiments:in vivo,Two-hybrid,Affinity Captur..."
7,P85B_HUMAN,5296,ERBB2_HUMAN,2064,0.89,"experiments:in vivo,Reconstituted Complex,Bioc..."
8,PTN18_HUMAN,26469,ERBB2_HUMAN,2064,0.88,"experiments:in vitro,anti tag coimmunoprecipit..."
9,LAP2_HUMAN,55914,ERBB2_HUMAN,2064,0.96,"experiments:in vitro,in vivo,Reconstituted Com..."


In [16]:
hippie_df.columns = ["source", "source_id", "target", "target_id", "weight", "evidence"]

In [45]:
hippie_df["source"] = [s.split("_")[0] for s in hippie_df["source"]]
hippie_df["target"] = [s.split("_")[0] for s in hippie_df["target"]]

In [46]:
hippie_df = hippie_df.dropna().drop_duplicates()

In [48]:
G =nx.from_pandas_dataframe(hippie_df, source="source", target="target", edge_attr="weight")
edges_to_remove = [(u, v) for (u, v, w) in list(G.edges(data=True)) if w["weight"] < 0.73]
G.remove_edges_from(edges_to_remove)

In [49]:
G = max(nx.connected_component_subgraphs(G), key=len)

In [50]:
nx.write_weighted_edgelist(G, path="../data/hippie/hippie_graph.edg", delimiter="\t")

In [51]:
!head ../data/hippie/hippie_graph.edg

RNF14	UB2E3	0.78
RNF14	UB2E2	0.79
RNF14	UB2E1	0.8
RNF14	ROA1	0.75
RNF14	UB2D4	0.73
RNF14	HNF1A	0.77
RNF14	UB2D1	0.73
RNF14	FA46A	0.73
RNF14	TAGL	0.75
RNF14	ANDR	0.9


In [3]:
pd.read_csv("../data/cora/cites.tsv", sep="\t")

,cited_paper_id,citing_paper_id
0,35,887
1,35,1033
2,35,1688
3,35,1956
4,35,8865
5,35,12576
6,35,15670
7,35,18582
8,35,27510
9,35,28290


In [41]:
with open("../data/cora/cites.tsv", "rb") as inf:
    next(inf, "")
    G = nx.read_edgelist(inf, delimiter="\t", )

In [42]:
len(G)

2708

In [43]:
nx.number_connected_components(G)

78

In [49]:
node_map = {n : i for i, n in enumerate(G.nodes())}

In [50]:
node_map

{u'74700': 0,
 u'684986': 1,
 u'262108': 454,
 u'23448': 4,
 u'131117': 5,
 u'144330': 6,
 u'193354': 7,
 u'1109199': 246,
 u'193352': 8,
 u'1121867': 9,
 u'1105505': 10,
 u'103543': 11,
 u'1104055': 12,
 u'469504': 13,
 u'6151': 16,
 u'6152': 17,
 u'189572': 18,
 u'6155': 19,
 u'43186': 167,
 u'287787': 21,
 u'289780': 22,
 u'277263': 187,
 u'1106287': 24,
 u'158098': 25,
 u'69392': 26,
 u'1132434': 27,
 u'69397': 28,
 u'65650': 29,
 u'65653': 30,
 u'8696': 2082,
 u'3237': 1817,
 u'100701': 31,
 u'1102442': 32,
 u'689439': 33,
 u'853114': 34,
 u'853115': 35,
 u'853116': 36,
 u'52003': 37,
 u'52000': 38,
 u'853118': 39,
 u'52007': 40,
 u'1129015': 41,
 u'1129018': 42,
 u'1132505': 43,
 u'43698': 44,
 u'436796': 1779,
 u'85324': 45,
 u'1113438': 46,
 u'166825': 2593,
 u'28851': 1363,
 u'63931': 49,
 u'93320': 50,
 u'990075': 51,
 u'2440': 52,
 u'644494': 53,
 u'1130568': 54,
 u'1109392': 55,
 u'1130567': 56,
 u'1106630': 57,
 u'1117184': 58,
 u'93923': 59,
 u'294145': 60,
 u'102939': 61

In [71]:
cora_labels = pd.read_csv("../data/cora/paper.tsv", sep="\t", )
cora_labels

,paper_id,class_label
0,35,Genetic_Algorithms
1,40,Genetic_Algorithms
2,114,Reinforcement_Learning
3,117,Reinforcement_Learning
4,128,Reinforcement_Learning
5,130,Reinforcement_Learning
6,164,Theory
7,288,Reinforcement_Learning
8,424,Rule_Learning
9,434,Reinforcement_Learning


In [70]:
label_map = {"Genetic_Algorithms" : 0, "Reinforcement_Learning" : 1, "Theory" : 2, 
            "Rule_Learning" : 3, "Case_Based" : 4, "Probabilistic_Methods" : 5, "Neural_Networks" : 6}

In [72]:
cora_label_matrix = lil_matrix((2708, 7))
for r in cora_labels.iterrows():
    paper_id = str(r[1]["paper_id"])
    paper_id = node_map[paper_id]
    class_label = r[1]["class_label"]
    class_label = label_map[class_label]
    cora_label_matrix[paper_id, class_label] = 1

In [73]:
cora_label_matrix = csr_matrix(cora_label_matrix)

In [74]:
sp.sparse.save_npz("../data/cora/paper_labels.npz", cora_label_matrix)

In [8]:
cora_content = pd.read_csv("../data/cora/content.tsv", sep="\t", )
cora_content

,paper_id,word_cited_id
0,35,word100
1,35,word1152
2,35,word1175
3,35,word1228
4,35,word1248
5,35,word1250
6,35,word1306
7,35,word141
8,35,word192
9,35,word20


In [10]:
import scipy as sp

In [57]:
from scipy.sparse import lil_matrix, csr_matrix

In [78]:
cora_content_matrix = lil_matrix((2708, 1433))

In [52]:
for r in cora_content.iterrows():
    paper_id = str(r[1]["paper_id"])
    paper_id = node_map[paper_id]
    word_cited_id = r[1]["word_cited_id"]
    word_cited_id = int(word_cited_id.split("word")[1]) - 1
    cora_content_matrix[paper_id, word_cited_id] = 1

In [58]:
cora_content_matrix = csr_matrix(cora_content_matrix)

In [65]:
sp.sparse.save_npz("../data/cora/cited_words.npz", cora_content_matrix)

In [68]:
sp.sparse.load_npz("../data/cora/cited_words.npz")

<2708x1433 sparse matrix of type '<type 'numpy.float64'>'
	with 53218 stored elements in Compressed Sparse Row format>